In [0]:
from delta.tables import DeltaTable

# Merge fro incremental updates
deltaTable = DeltaTable.forPath(spark, "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv")
updates = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct_updated.csv", header=True , inferSchema = True)

In [0]:
# Read the October 2019 CSV file as a Spark DataFrame
oct_2019_df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)

# The duplicate source DataFrame to avoid multiple matches per target row
oct_2019_df = oct_2019_df.dropDuplicates (["event_time", "user_id", "product_id"])

# Perform incremental MERGE into the Delta table
from delta.tables import DeltaTable

delta_table = DeltaTable.forName(spark, "workspace.default.events_table")

delta_table.alias("target").merge(
    oct_2019_df.alias("source"),
    "target.event_time = source.event_time AND target.user_id = source.user_id AND target.product_id = source.product_id"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

In [0]:
display(spark.sql("DESCRIBE HISTORY workspace.default.events_table"))

In [0]:
# Time travel using the Delta table name  

v0 = spark.read.format("delta").option("versionAsOf", 1).table("workspace.default.events_table")

history_timestamp = spark.read.format('delta').option("timestampAsOf","2026-01-14 06:51:19.0").table("workspace.default.events_table")
display(history_timestamp)


In [0]:
spark.sql("OPTIMIZE workspace.default.events_table ZORDER BY (event_type, user_id)")

In [0]:
spark.sql("VACUUM workspace.default.events_table RETAIN 200 HOURS")